# OpenAI Quickstart

# Overview  
"Large language models are functions that map text to text. Given an input string of text, a large language model tries to predict the text that will come next"(1). This "quickstart" notebook will introduce users to high-level LLM concepts, core package requirements for getting started with AML, an soft introduction to prompt design, and severa short examples of different use cases.  

For more quickstart examples please refer to the official Azure Open AI Quickstart Documentation https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio

## Table of Contents  

[Overview](#overview)  
[How to use OpenAI Service](#how-to-use-openai-service)  
[1. Creating your OpenAI Service](#1.-creating-your-openai-service)  
[2. Installation](#2.-installation)    
[3. Credentials](#3.-credentials)  

[Use Cases](#use-cases)    
[1. Summarize Text](#1.-summarize-text)  
[2. Classify Text](#2.-classify-text)  
[3. Generate New Product Names](#3.-generate-new-product-names)  
[4. Fine Tune a Classifier](#4.fine-tune-a-classifier)  
[5. Embeddings!]((#5.-embeddings!))

[References](#references)

### Getting started with Azure OpenAI Service

New customers will need to [apply for access](https://aka.ms/oai/access) to Azure OpenAI Service.  
After approval is complete, customers can log into the Azure portal, create an Azure OpenAI Service resource, and start experimenting with models via the studio  

[Great resource for getting started quickly](https://techcommunity.microsoft.com/t5/educator-developer-blog/azure-openai-is-now-generally-available/ba-p/3719177 )


### Build your first prompt  
This short exercise will provide a basic introduction for submitting prompts to an OpenAI model for a simple task "summarization".  

![](images/generative-AI-models-reduced.jpg)  


**Steps**:  
1. Install OpenAI library in your python environment  
2. Load standard helper libraries and set your typical OpenAI security credentials for the OpenAI Service that you've created  
3. Choose a model for your task  
4. Create a simple prompt for the model  
5. Submit your request to the model API!

### 1. Import helper libraries and instantiate credentials

In [27]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
# load_dotenv()


In [28]:
load_dotenv("../config/environment_variables.env")

True

In [6]:

client = AzureOpenAI(
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT_TEXT"), 
  api_key=os.environ.get("AZURE_OPENAI_KEY_TEXT"),  
  api_version=os.environ.get("AZURE_OPENAI_API_VERSION")
)

### 2. Finding the right model  
The GPT-3.5-turbi or GPT-4 models can understand and generate natural language. The service offers four model capabilities, each with different levels of power and speed suitable for different tasks. 

[Azure OpenAI models](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models)  
![](images/a-b-c-d-models-reduced.jpg)  


In [29]:
# Select the General Purpose curie model for text
model = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT")
modelgpt4 = os.environ.get("AZURE_OPENAI_GPT4_DEPLOYMENT")
modelembeddings = os.environ.get("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

### 3. Prompt Design  

"The magic of large language models is that by being trained to minimize this prediction error over vast quantities of text, the models end up learning concepts useful for these predictions. For example, they learn concepts like"(1):

* how to spell
* how grammar works
* how to paraphrase
* how to answer questions
* how to hold a conversation
* how to write in many languages
* how to code
* etc.

#### How to control a large language model  
"Of all the inputs to a large language model, by far the most influential is the text prompt(1).

Large language models can be prompted to produce output in a few ways:

Instruction: Tell the model what you want
Completion: Induce the model to complete the beginning of what you want
Demonstration: Show the model what you want, with either:
A few examples in the prompt
Many hundreds or thousands of examples in a fine-tuning training dataset"



#### There are three basic guidelines to creating prompts:

**Show and tell**. Make it clear what you want either through instructions, examples, or a combination of the two. If you want the model to rank a list of items in alphabetical order or to classify a paragraph by sentiment, show it that's what you want.

**Provide quality data**. If you're trying to build a classifier or get the model to follow a pattern, make sure that there are enough examples. Be sure to proofread your examples — the model is usually smart enough to see through basic spelling mistakes and give you a response, but it also might assume this is intentional and it can affect the response.

**Check your settings.** The temperature and top_p settings control how deterministic the model is in generating a response. If you're asking it for a response where there's only one right answer, then you'd want to set these lower. If you're looking for more diverse responses, then you might want to set them higher. The number one mistake people use with these settings is assuming that they're "cleverness" or "creativity" controls.


Source: https://github.com/Azure/OpenAI/blob/main/How%20to/Completions.md

![](images/prompt_design.jpg)
image is creating your first text prompt!

### 4. Submit!

In [8]:
# Create your first prompt
text_prompt = "Saya adalah warga negara indonesia. Beritahu saya secara singkat dalam 3 kalimat. Apa makna dari bendera merah putih."

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])


In [9]:
print(response.choices[0].message.content)

Bendera merah putih memiliki makna yang mendalam bagi warga negara Indonesia. Merah melambangkan keberanian dan semangat, sementara putih melambangkan kesucian dan perdamaian, menggambarkan semangat persatuan dan kesatuan bangsa Indonesia.


### Repeat the same call, how do the results compare?

In [10]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

print(response.choices[0].message.content)

Bendera merah putih adalah simbol kebanggaan dan identitas bangsa Indonesia. Warna merah melambangkan keberanian dan semangat, sedangkan putih melambangkan kejujuran dan kemurnian. Bendera ini menggambarkan semangat persatuan dan kesatuan serta nilai-nilai nasionalisme bagi rakyat Indonesia.


## Summarize Text  
#### Challenge  
Summarize text by adding a 'tl;dr:' to the end of a text passage. Notice how the model understands how to perform a number of tasks with no additional instructions. You can experiment with more descriptive prompts than tl;dr to modify the model’s behavior and customize the summarization you receive(3).  

Contoh Prompt : 

Menkeu Terima Kunjungan Satu Kahkonen, World Bank Country Director for Indonesia and Timor-Leste\nJakarta, 20/3/2024 Kemenkeu – Menteri Keuangan (Menkeu) Sri Mulyani Indrawati menerima kunjungan dari Country Director World Bank untuk Indonesia dan Timor Leste, Satu Kahkonen di Jakarta sore ini, Rabu (20/03). Sejak Februari 2020, Satu menjadi Country Director dan mengawasi program World Bank di kedua negara tersebut.\nPada pertemuan tersebut, Menteri Sri Mulyani bersama dengan Satu Kahkonen berkesempatan membahas banyak hal, utamanya mengenai dukungan World Bank untuk Indonesia.\n”Kami berbincang mengenai beragam kerja sama dan program World Bank di Indonesia. Diantaranya, isu-isu terkait transisi energi - mulai mekanisme transisi energi hingga Just Energy Transition Partnership - dan perubahan iklim,” ungkap Menkeu dikutip dari laman instagram resmi @smindrawati.\nPada kesempatan yang sama, Menkeu bersama dengan Satu Kahkonen juga membahas mengenai program-program International Bank for Reconstruction and Development (IBRD), termasuk kerja sama dalam Program Keluarga Harapan.\n“Salah satu program kerja sama yang membantu meredam dampak-dampak negatif penyebaran virus Covid-19,” ungkapnya.\nTerakhir, Satu juga menyampaikan salam perpisahannya karena akan ditugaskan di Afrika Selatan pada bulan mendatang.\n”Thank you for the cooperation, warmth, and partnership, Satu! Have a good journey in your next assignment..!,” tukas Menkeu. (ig/feb/al)


Tl;dr

# Exercises for several use cases  
1. Summarize Text  
2. Classify Text  
3. Generate New Product Names
4. Embeddings
5. Fine tune a classifier

In [11]:
prompt = "Menkeu Terima Kunjungan Satu Kahkonen, World Bank Country Director for Indonesia and Timor-Leste\nJakarta, 20/3/2024 Kemenkeu – Menteri Keuangan (Menkeu) Sri Mulyani Indrawati menerima kunjungan dari Country Director World Bank untuk Indonesia dan Timor Leste, Satu Kahkonen di Jakarta sore ini, Rabu (20/03). Sejak Februari 2020, Satu menjadi Country Director dan mengawasi program World Bank di kedua negara tersebut.\nPada pertemuan tersebut, Menteri Sri Mulyani bersama dengan Satu Kahkonen berkesempatan membahas banyak hal, utamanya mengenai dukungan World Bank untuk Indonesia.\n”Kami berbincang mengenai beragam kerja sama dan program World Bank di Indonesia. Diantaranya, isu-isu terkait transisi energi - mulai mekanisme transisi energi hingga Just Energy Transition Partnership - dan perubahan iklim,” ungkap Menkeu dikutip dari laman instagram resmi @smindrawati.\nPada kesempatan yang sama, Menkeu bersama dengan Satu Kahkonen juga membahas mengenai program-program International Bank for Reconstruction and Development (IBRD), termasuk kerja sama dalam Program Keluarga Harapan.\n“Salah satu program kerja sama yang membantu meredam dampak-dampak negatif penyebaran virus Covid-19,” ungkapnya.\nTerakhir, Satu juga menyampaikan salam perpisahannya karena akan ditugaskan di Afrika Selatan pada bulan mendatang.\n”Thank you for the cooperation, warmth, and partnership, Satu! Have a good journey in your next assignment..!,” tukas Menkeu. (ig/feb/al)\n\nTl;dr"


In [13]:
#Setting a few additional, typical parameters during API Call
#gpt-3.5-turbo

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

print(response.choices[0].message.content)

Menteri Keuangan Sri Mulyani Indrawati receives a visit from Satu Kahkonen, the World Bank Country Director for Indonesia and Timor-Leste. They discuss various cooperation programs and issues related to energy transition, climate change, and the International Bank for Reconstruction and Development. Satu also mentions that they will be assigned to South Africa in the coming month.


In [31]:
#Setting a few additional, typical parameters during API Call
#gpt-4

response = client.chat.completions.create(
  model=modelgpt4,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

print(response.choices[0].message.content)

Product names: AdaptEase, UniFit, OmniSteps, FlexFoot, UniversalStride.


## Classify Text  
#### Challenge  
Classify items into categories provided at inference time. In the following example we provide both the categories and the text to classify in the prompt(*playground_reference). 

Customer Inquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:

Classified category:


In [15]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]

inquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:

Classified category:


In [16]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
       model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

'Hardware Support'

## Generate New Product Names
#### Challenge
Create product names from examples words. Here we include in the prompt information about the product we are going to generate names for. We also provide a similar example to show the pattern we wish to receive. We have also set the temperature value high to increase randomness and more innovative responses.

Product description: A home milkshake maker
Seed words: fast, healthy, compact.
Product names: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Product description: A pair of shoes that can fit any foot size.
Seed words: adaptable, fit, omni-fit.

In [17]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

Product description: A home milkshake maker
Seed words: fast, healthy, compact.
Product names: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Product description: A pair of shoes that can fit any foot size.
Seed words: adaptable, fit, omni-fit.


In [18]:
#Setting a few additional, typical parameters during API Call
response = client.chat.completions.create(
       model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},]
)

response.choices[0].message.content

'Product names: FlexFit, PerfectFit, OmniShoe, AllFit'

## Embeddings!  
This section will show how to retrieve embeddings, and find similarity between words, sentences, and documents

model = 'text-embedding-ada-002'

In [32]:
import numpy as np

def cosine_similarity(query_embedding, embeddings, distance_metric='cosine'):
    if distance_metric == 'cosine':
        distances = np.dot(embeddings, query_embedding) / (np.linalg.norm(embeddings) * np.linalg.norm(query_embedding))
        distances = 1 - distances  
    else:
        raise ValueError("Metrik (fungsi jarak) tidak didukung. Gunakan 'cosine'.")

    return distances

In [34]:
text_1 = 'Pohon hijau berdiri di tepi sungai yang tenang.'
emb_text_1 = client.embeddings.create(input=[text_1], model=modelembeddings).data[0].embedding
print(emb_text_1)


[0.023568056523799896, -0.015377561561763287, 0.017208820208907127, 0.006798236630856991, -0.013358158990740776, 0.030002551153302193, -0.012379814870655537, 0.001182949636131525, -0.011620971374213696, -0.015427732840180397, 0.010586184449493885, 0.01255541481077671, -0.005048505961894989, -0.011156884953379631, 0.0032047037966549397, -0.016343362629413605, 0.03471866995096207, -0.030178150162100792, 0.01818716526031494, 0.007557080592960119, -0.0018516415730118752, 0.01865125074982643, 0.0002939735713880509, 0.019265852868556976, -0.005659971386194229, 0.018061736598610878, 0.00353709002956748, -0.017773250117897987, 0.012580500915646553, 0.004966977518051863, 0.020482510328292847, -0.018613623455166817, 0.004139147698879242, 0.008529153652489185, -0.011827928945422173, -0.015791475772857666, -0.0019943166989833117, -0.0025054388679564, 0.007996081374585629, -0.009018325246870518, 0.019993338733911514, -0.012216757982969284, 0.003203136147931218, -0.021122196689248085, -0.00051543366

In [35]:
text2 = 'Ayahku membuat bagan pohon keluarga di tembok yang hijau'
emb_text_2 = client.embeddings.create(input=[text2], model=modelembeddings).data[0].embedding

In [36]:
print(cosine_similarity(emb_text_1, emb_text_2))

0.1137047439703619


In [40]:
text3 = 'Pohon dan tumbuhan hidup di sekitar sungai Nil.'
emb_text_3 = client.embeddings.create(input=[text3], model=modelembeddings).data[0].embedding

In [41]:
print(cosine_similarity(emb_text_1, emb_text_3))

0.09541124818987201


In [39]:
# compare several words
automobile_embedding    = client.embeddings.create(input='automobile', model=modelembeddings).data[0].embedding
vehicle_embedding       = client.embeddings.create(input='vehicle', model=modelembeddings).data[0].embedding
dinosaur_embedding      = client.embeddings.create(input='dinosaur', model=modelembeddings).data[0].embedding
stick_embedding         = client.embeddings.create(input='stick', model=modelembeddings).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

0.08382373906311491
0.1665570317722117
0.21796415287146154


## Comparing article from cnn daily news dataset
source: https://huggingface.co/datasets/cnn_dailymail


In [42]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"' 
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()                      

,articles,highligths
0,"BREMEN, Germany -- Carlos Alberto, who scored ...",Werder Bremen pay a club record $10.7 million ...
1,"(CNN) -- Football superstar, celebrity, fashio...",Beckham has agreed to a five-year contract wit...
2,"LOS ANGELES, California (CNN) -- Youssif, the ...","Boy on meeting Spider-Man: ""It was my favorite..."


In [43]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=modelembeddings).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=modelembeddings).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=modelembeddings).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

0.2378745657639586
0.2896765175077112


# References  
-Azure Reference Documentation  
-Azure OpenAI GitHub Repo
-cookbooks  
-OpenAI website  

1 - [Openai Cookbook](https://github.com/openai/openai-cookbook)  
2 - [Azure Documentation - Azure Open AI Models](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models)  
3 - [OpenAI Studio Examples](https://oai.azure.com/portal)  
4 - [[PUBLIC] Best practices for fine-tuning GPT-3 to classify text](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#)

# For More Help  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)  
AI Specialized CSAs [aka.ms/airangers](aka.ms/airangers)